In [ ]:
import keras
import numpy as np
import pandas as pd
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import mlflow
from mlflow.models import infer_signature

##load the dataset
data = pd.read_csv(
    "https://ram.githubusercontent.com/mlflow/mlflow/master/datasets/winequality-white.csv",
    sep=";",
)
#split the data into training, validation and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)

train_x=train.drop(['quality'], axis=1).values
train_y = train['quality'].values.ravel()
test_x=test.drop(['quality'], axis=1).values
test_y=test['quality'].values.ravel()

train_x, valid_x, train_y, valid_y=train_test_split(train_x, train_y, test_size=0.2, random_state=42)
signature=infer_signature(train_x, train_y)

#ANN Model

def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
  #Define Model Archicteture
  mean=np.mean(train_x, axis=0)
  var=np.var(train_x, axis=0)
  std=np.std(train_x, axis=0)

  model = keras.Sequential([
      keras.Input(shape=(train_x.shape[1],)),
      keras.layers.Normalization(mean=mean, variance=var),
      keras.layers.Dense(64, activation='relu'),
      keras.layers.Dense(1)

  ])

  #compile the model
  model.compile(optimizer=keras.optimizers.SGD(
      learning_rate=params["lr"], momentum=params["momentum"]),
                loss='mean_squared_error',
                metrics=[keras.metreics.RootMeanSquaredError()])

  ##train the ANN Model with lr and momentum params with MLFLOW Tracking
  with mlflow.start_run(nested=true):
    mlflow.fit(train_x,train_y, epochs=epochs, validation_data=(valid_x, valid_y,batch_size=64))
    eval_result = model.evaluate(valid_x, valid_y, batch_size=64)

    eval_rme=eval_result[1]

    #Log the parameters and results

    mlflow.log_params(params)
    mlflow.log_metric("eval_rmse", eval_rme)

    #log the model
    mlflow.tensorflow.log_model(model, "model", signature=signature)

    return {'loss': eval_rme, 'status': STATUS_OK, 'model': model}


In [ ]:
def objective(params):
  #Mlflow will track
  result = train_model(
      params,
      epochs=3,
      train_x=train_x,
      train_y=train_y,
      valid_x=valid_x,
      valid_y=valid_y,
      test_x=test_x,
      test_y=test_y,
  )
  return result

In [ ]:
space={
    'lr': hp.loguniform('lr', np.log(1e-5), np.log(1e-1)),
    'momentum': hp.uniform('momentum', 0.0, 1.0)
}

In [ ]:
mlflow.set_experiment("/wine_quality")
with mlflow.start_run():
  #Conduct the hyperparameter search using hyperopt
  trials=Trials()
  best=fmin(
      fn=objective,
      space=space,
      algo=tpe.suggest,
      max_evals=4
  )

  #fetch the details of the best run
  best_run=sorted(trials.results, key=lambda x: x['loss'])[0]
  best_run